# Resources

1. Chapter 9-- the book.
2. The web-links in this notebook.

# Objectives

1. Understanding the main concepts of algorithms and metrics.
2. Understanding the algorthematic logic in details:
    - K-means.
    - HC.
    - DBSAN and HDBSAN.

# Clustering algorithms

1. Partitioning models: 
    - K-means vs. K-modiods.
2. Hierarchical models:
    - HC
3. [Subspace/projected models](https://en.wikipedia.org/wiki/Clustering_high-dimensional_data#Subspace_clustering): High-dimensionality space.
4. Mixture models: Mixure space.
    - [Guisian Mixure models (Expectation Maximization (EM) model)](https://towardsdatascience.com/gaussian-mixture-models-d13a5e915c8e)
5. Graph-based models: Spatial (Corrdinate-based/relationship) space.
    - [Spectral models](https://towardsdatascience.com/spectral-clustering-aba2640c0d5b#:~:text=Spectral%20clustering%20is%20a%20technique,non%20graph%20data%20as%20well)
6. Density models: Condense/(Noise or sparse) space
    - [DBSCAN-Density-Based Spatial Clustering of Applications with Noise](https://www.kdnuggets.com/2020/04/dbscan-clustering-algorithm-machine-learning.html)
        - [OPTICS-Ordering points to identify the clustering structure](https://xzz201920.medium.com/optics-d80b41fd042a)
    - HDBSAN
7. Others

# [Availablity In scikit-learn](https://scikit-learn.org/stable/modules/clustering.html)

# Metrics in Scikit-learn

In [ ]:
from sklearn import metrics

In [ ]:
help(metrics)

In [ ]:
help(metrics.homogeneity_completeness_v_measure)

# [Graph-based clustering](https://web.iitd.ac.in/~bspanda/graphclustering.pdf)

1. Graph-Based clustering uses the proximity graph
    - Start with the proximity matrix
    - Consider each point as a node in a graph
    - Each edge between two nodes has a weight which is the proximity between the two points
    - Initially the proximity graph is fully connected
    - MIN (single-link) and MAX (complete-link) can be viewed as starting with this graph
2. In the simplest case, clusters are connected components in the graph.

## Minumum spanning tree

<img style="float:center" src="./images/ST.png" alt="drawing" height="600" width="600"/>

<img style="float:center" src="./images/MST.png" alt="drawing" height="600" width="600"/>

## K-Spanning tree clustering

<img style="float:center" src="./images/KST.png" alt="drawing" height="600" width="600"/>

# Density-based clustering

Density-Based Clustering refers to  unsupervised learning methods that identify distinctive groups/clusters in the data, based on the idea that a cluster in a data space is a contiguous region of high point density, separated from other such clusters by contiguous regions of low point density. The data points in the separating regions of low point density are typically considered noise/outliers.

## Density-Based Spatial Clustering of Applications with Noise (DBSCAN)

### Parameters

<b> minPts: </b> The minimum number of points (a threshold) clustered together for a region to be considered dense.  
<b> eps (ε): </b> A distance measure that will be used to locate the points in the neighborhood of any point.  


### Concepts

<b> Reachability </b> in terms of density establishes a point to be reachable from another if it lies within a particular distance (eps) from it.

<b> Connectivity </b>, on the other hand, involves a transitivity based chaining-approach to determine whether points are located in a particular cluster. For example, p and q points could be connected if p->r->s->t->q, where a->b means b is in the neighborhood of a.

<img style="float:center" src="./images/DBSCAN.png" alt="drawing" height="600" width="600"/>


### Parameter estimation: [Link](https://www.kdnuggets.com/2020/04/dbscan-clustering-algorithm-machine-learning.html)


### [Algorithm](https://www.machinecurve.com/index.php/2020/12/09/performing-dbscan-clustering-with-python-and-scikit-learn/)

1. We set values for 𝜖 and minPts.
2. We randomly select a point from the samples that has not been checked before.
3. We retrieve the 𝜖−neighborhood for this point. If it equals or exceeds minPts, we signal it as a cluster. Otherwise, we label it as noise.
4. We signal the 𝜖−neighborhood as being part of the cluster. This means that for each point of that neighborhood, its own 𝜖−neighborhood is added to the cluster as well, and so on, and so on. We continue until no further point can be added to the cluster. Note that the point originally labeled as noise can now also become part of this cluster (it may be part of the 𝜖−neighborhood of one of the other points), or of another cluster later, because:
5. We now start at (2) again, unless all points have been checked and labeled.

### DBSCAN python implementation using sklearn

In [ ]:
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler

# Generate sample data
centers = [[1, 1], [-1, -1], [1, -1]]
X, labels_true = make_blobs(n_samples=750, centers=centers, cluster_std=0.4,
                            random_state=1)

X = StandardScaler().fit_transform(X)

# Compute DBSCAN
db = DBSCAN(eps=0.3, min_samples=10).fit(X)
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print('Estimated number of clusters: %d' % n_clusters_)
print('Estimated number of noise points: %d' % n_noise_)
print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels_true, labels))
print("Completeness: %0.3f" % metrics.completeness_score(labels_true, labels))
print("V-measure: %0.3f" % metrics.v_measure_score(labels_true, labels))
print("Adjusted Rand Index: %0.3f"
      % metrics.adjusted_rand_score(labels_true, labels))
print("Adjusted Mutual Information: %0.3f"
      % metrics.adjusted_mutual_info_score(labels_true, labels))
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X, labels))

# Plot result
import matplotlib.pyplot as plt
%matplotlib inline

# Black removed and is used for noise instead.
unique_labels = set(labels)
colors = [plt.cm.Spectral(each)
          for each in np.linspace(0, 1, len(unique_labels))]
for k, col in zip(unique_labels, colors):
    if k == -1:
        # Black used for noise.
        col = [0, 0, 0, 1]

    class_member_mask = (labels == k)

    xy = X[class_member_mask & core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
             markeredgecolor='k', markersize=14)

    xy = X[class_member_mask & ~core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
             markeredgecolor='k', markersize=6)

plt.title('Estimated number of clusters: %d' % n_clusters_)
plt.show()

In [ ]:
#Non-spherical data

import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.datasets import make_circles
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
X, y = make_circles(n_samples=750, factor=0.3, noise=0.1,random_state=1)
X = StandardScaler().fit_transform(X)
db=DBSCAN(eps=0.3, min_samples=10)
y_pred = db.fit_predict(X)
labels = db.labels_

plt.scatter(X[:,0], X[:,1], c=y_pred)
print('Number of clusters: {}'.format(len(set(y_pred[np.where(y_pred != -1)]))))
print('Homogeneity: {}'.format(metrics.homogeneity_score(y, y_pred)))
print('Completeness: {}'.format(metrics.completeness_score(y, y_pred)))
print("V-measure: %0.7f" % metrics.v_measure_score(y, labels))
print("Adjusted Rand Index: %0.7f"
      % metrics.adjusted_rand_score(y, labels))
print("Adjusted Mutual Information: %0.7f"
      % metrics.adjusted_mutual_info_score(y, labels))
print("Silhouette Coefficient: %0.7f"
      % metrics.silhouette_score(X, labels))

In [ ]:
#K-means

import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.datasets import make_circles
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
km=KMeans(n_clusters=2,random_state=1)
y_pred = km.fit_predict(X)
labels = km.labels_

plt.scatter(X[:,0], X[:,1], c=y_pred)
print('Number of clusters: {}'.format(len(set(y_pred[np.where(y_pred != -1)]))))
print('Homogeneity: {}'.format(metrics.homogeneity_score(y, y_pred)))
print('Completeness: {}'.format(metrics.completeness_score(y, y_pred)))
print("V-measure: %0.7f" % metrics.v_measure_score(y, labels))
print("Adjusted Rand Index: %0.7f"
      % metrics.adjusted_rand_score(y, labels))
print("Adjusted Mutual Information: %0.7f"
      % metrics.adjusted_mutual_info_score(y, labels))
print("Silhouette Coefficient: %0.7f"
      % metrics.silhouette_score(X, labels))

## [OPTICS-Ordering points to identify the clustering structure](https://xzz201920.medium.com/optics-d80b41fd042a)

The basic idea is similar to DBSCAN, but it addresses one of DBSCAN’s major weaknesses: the problem of detecting meaningful clusters in data of varying density.       

To do so, the points of the database are (linearly) ordered such that spatially closest points become neighbors in the ordering. Additionally, a special distance is stored for each point that represents the density that must be accepted for a cluster so that both points belong to the same cluster. This is represented as a dendrogram.

## [HDBSCAN](https://hdbscan.readthedocs.io/en/latest/how_hdbscan_works.html#transform-the-space)

It extends DBSCAN by converting it into a hierarchical clustering algorithm, and then using a technique to extract a flat clustering based in the stability of clusters.


### Algorithm

1. Transform the space according to the density/sparsity.
2. Build the minimum spanning tree of the distance weighted graph.
3. Construct a cluster hierarchy of connected components.
4. Condense the cluster hierarchy based on minimum cluster size.
5. Extract the stable clusters from the condensed tree.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.datasets as data
%matplotlib inline
sns.set_context('poster')
sns.set_style('white')
sns.set_color_codes()
plot_kwds = {'alpha' : 0.5, 's' : 80, 'linewidths':0}

In [ ]:
moons, _ = data.make_moons(n_samples=50, noise=0.05)
blobs, _ = data.make_blobs(n_samples=50, centers=[(-0.75,2.25), (1.0, 2.0)], cluster_std=0.25)
test_data = np.vstack([moons, blobs])
plt.scatter(test_data.T[0], test_data.T[1], color='b', **plot_kwds)

In [ ]:
import hdbscan
help (hdbscan)

In [ ]:
clusterer = hdbscan.HDBSCAN(min_cluster_size=5, gen_min_span_tree=True)
clusterer.fit(test_data)

#### Transform the space according to the density/sparsity.

It uses DBSCAN to find the islands of higher density amid a sea of sparser noise by finding core and reachable distances.

#### Build the minimum spanning tree of the distance weighted graph.

Now consider a threshold value, starting high, and steadily being lowered. Drop any edges with weight above that threshold. As we drop edges we will start to disconnect the graph into connected components. Eventually we will have a hierarchy of connected components (from completely connected to completely disconnected) at varying threshold levels.

In [ ]:
clusterer.minimum_spanning_tree_.plot(edge_cmap='viridis',
                                      edge_alpha=0.6,
                                      node_size=80,
                                      edge_linewidth=2)

#### Construct a cluster hierarchy of connected components.

Given the minimal spanning tree, the next step is to convert that into the hierarchy of connected components. This is most easily done in the reverse order: sort the edges of the tree by distance (in increasing order) and then iterate through, creating a new merged cluster for each edge.  

Remeber single-linkage approach is to link pairs with the closest values.

In [ ]:
clusterer.single_linkage_tree_.plot(cmap='viridis', colorbar=True)

#### Condense the cluster hierarchy based on minimum cluster size.

Given:  


We need to merge clusters such that each cluster has the size determined by the cluster size paramter.

In [ ]:
clusterer.condensed_tree_.plot()

#### Extract the stable clusters from the condensed tree.

For a given cluster we can then define values $\lambda_{\mathrm{birth}}$ and $\lambda_{\mathrm{death}}$ to be the lambda value when the cluster split off and became it’s own cluster, and the lambda value (if any) when the cluster split into smaller clusters respectively. 

In turn, for a given cluster, for each point p in that cluster we can define the value $\lambda_p$ as the lambda value at which that point ‘fell out of the cluster’ which is a value somewhere between $\lambda_{\mathrm{birth}}$ and $\lambda_{\mathrm{death}}$ since the point either falls out of the cluster at some point in the cluster’s lifetime, or leaves the cluster when the cluster splits into two smaller clusters.Now, for each cluster compute the stability as




Declare all leaf nodes to be selected clusters. Now work up through the tree (the reverse topological sort order). If the sum of the stabilities of the child clusters is greater than the stability of the cluster, then we set the cluster stability to be the sum of the child stabilities. If, on the other hand, the cluster’s stability is greater than the sum of its children then we declare the cluster to be a selected cluster and unselect all its descendants. Once we reach the root node we call the current set of selected clusters our flat clustering and return that.

In [ ]:
clusterer.condensed_tree_.plot(select_clusters=True, selection_palette=sns.color_palette())

#### Plot the resultant clusters

In [ ]:
palette = sns.color_palette()
cluster_colors = [sns.desaturate(palette[col], sat)
                  if col >= 0 else (0.5, 0.5, 0.5) for col, sat in
                  zip(clusterer.labels_, clusterer.probabilities_)]
plt.scatter(test_data.T[0], test_data.T[1], c=cluster_colors, **plot_kwds)